# Fine-tuning a model with the Trainer API

Install the Transformers, Datasets, and Evaluate libraries to run this notebook.

In [ ]:
# Install required libraries for Transformers, datasets, and evaluation
!uv pip install datasets evaluate transformers[sentencepiece]

In [ ]:
# Set up the complete pipeline for fine-tuning with Trainer API
# Load and preprocess the MRPC dataset from GLUE benchmark
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

raw_datasets = load_dataset("glue", "mrpc")
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

# Define tokenization function for sentence pairs
def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)

# Apply tokenization to all dataset splits
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
# Set up data collator for dynamic padding during training
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
# Configure basic training arguments
# "test-trainer" is the output directory where model checkpoints will be saved
from transformers import TrainingArguments

training_args = TrainingArguments("test-trainer")

In [ ]:
# Load pre-trained model for sequence classification
# num_labels=2 because MRPC is a binary classification task (equivalent/not equivalent)
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

In [ ]:
# Initialize the Trainer with model, arguments, and datasets
# Trainer handles the training loop, evaluation, and logging automatically
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    processing_class=tokenizer,  # Updated parameter name for tokenizer
)

In [ ]:
# Start the training process
# This will fine-tune the pre-trained BERT model on the MRPC dataset
trainer.train()

In [ ]:
# Generate predictions on the validation set
# predictions.predictions contains raw logits, predictions.label_ids contains true labels
predictions = trainer.predict(tokenized_datasets["validation"])
print(predictions.predictions.shape, predictions.label_ids.shape)

In [ ]:
# Convert logits to predicted class labels
# argmax finds the class with highest probability for each example
import numpy as np

preds = np.argmax(predictions.predictions, axis=-1)

In [ ]:
# Evaluate model performance using GLUE metrics
# For MRPC task, this returns accuracy and F1 score
import evaluate

metric = evaluate.load("glue", "mrpc")
metric.compute(predictions=preds, references=predictions.label_ids)

In [ ]:
# Define a function to compute metrics during training
# This function will be called automatically during evaluation
def compute_metrics(eval_preds):
    metric = evaluate.load("glue", "mrpc")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
# Set up training with automatic evaluation after each epoch
# evaluation_strategy="epoch" runs evaluation at the end of each training epoch
training_args = TrainingArguments("test-trainer", evaluation_strategy="epoch")
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    processing_class=tokenizer,
    compute_metrics=compute_metrics,  # Add metrics computation during training
)

In [ ]:
# Train the model with evaluation after each epoch
# This will show accuracy and F1 scores during training
trainer.train()